In [ ]:
# 환경 설정 및 라이브러리 임포트
import os
import sys
import re
import time
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Optional, Dict, Tuple
from collections import Counter


# LangChain & Upstage
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader

# Wikipedia (지정된 API만 사용)
import wikipediaapi

# API Key 설정
os.environ.setdefault("UPSTAGE_API_KEY", "up_g01lLro3nefoqy9IZGw3NK4ExjfIQ") # 본인의 키로 대체하세요
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY", "")

# 작업 디렉토리 설정
CURRENT_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
if Path.cwd() != CURRENT_DIR:
    os.chdir(CURRENT_DIR)

print(f"✅ 작업 디렉토리: {CURRENT_DIR}")

In [ ]:
# ==================================================================
# 1. 이화여대 학칙 데이터 로딩 (PDF & CSV)
# ==================================================================

def split_by_articles(text: str, is_appendix: bool = False) -> List[str]:
    """
    PDF 텍스트를 조항별로 정확히 분리하는 함수
    이화여대 학칙 구조 분석:
    - 별표 1, 별표 2, 별표 3 등 (표)
    - 제1조, 제2조, 제3조 등 (조항)
    - ①, ②, ③ 등 (항목)
    - 부칙 (부칙 시작)
    """
    # 조항 시작 패턴 (우선순위 순서대로)
    # 1. 별표 (표는 독립적인 단위)
    # 2. 제X조 (조항)
    # 3. ①항, ②항 등 (항목 - 조항 내 세부사항)
    
    splits = []
    
    # 패턴 1: 별표로 시작하는 부분 (가장 큰 단위)
    star_pattern = r'(별표\s*\d+)'
    star_matches = list(re.finditer(star_pattern, text))
    
    # 패턴 2: 제X조로 시작하는 부분
    article_pattern = r'(제\d+조(?:\s*\([^)]+\))?)'  # 제1조, 제1조(목적) 등
    article_matches = list(re.finditer(article_pattern, text))
    
    # 별표가 있으면 별표 기준으로 먼저 분리
    if star_matches:
        last_pos = 0
        for match in star_matches:
            pos = match.start()
            if pos > last_pos:
                chunk = text[last_pos:pos].strip()
                if chunk:
                    splits.append(chunk)
            last_pos = pos
        # 마지막 별표 이후 텍스트
        if last_pos < len(text):
            chunk = text[last_pos:].strip()
            if chunk:
                splits.append(chunk)
        
        # 별표로 분리된 각 청크를 제X조로 다시 분리
        refined_splits = []
        for split in splits:
            split_article_matches = list(re.finditer(article_pattern, split))
            if split_article_matches:
                split_last_pos = 0
                for match in split_article_matches:
                    pos = match.start()
                    if pos > split_last_pos:
                        chunk = split[split_last_pos:pos].strip()
                        if chunk:
                            refined_splits.append(chunk)
                    split_last_pos = pos
                if split_last_pos < len(split):
                    chunk = split[split_last_pos:].strip()
                    if chunk:
                        refined_splits.append(chunk)
            else:
                refined_splits.append(split)
        splits = refined_splits
    elif article_matches:
        # 별표가 없으면 제X조 기준으로 분리
        last_pos = 0
        for match in article_matches:
            pos = match.start()
            if pos > last_pos:
                chunk = text[last_pos:pos].strip()
                if chunk:
                    splits.append(chunk)
            last_pos = pos
        # 마지막 조항 이후 텍스트
        if last_pos < len(text):
            chunk = text[last_pos:].strip()
            if chunk:
                splits.append(chunk)
    else:
        # 별표도 제X조도 없으면 전체를 하나의 청크로
        splits = [text]
    
    # 부칙의 경우 "부칙" 키워드로도 분리
    if is_appendix and splits:
        final_splits = []
        for split in splits:
            # "부칙" 키워드가 중간에 나오면 분리
            if "부칙" in split:
                parts = re.split(r'(부칙)', split, maxsplit=1)
                if len(parts) > 1:
                    if parts[0].strip():
                        final_splits.append(parts[0].strip())
                    if len(parts) > 2:
                        final_splits.append(parts[1] + parts[2].strip())
                else:
                    final_splits.append(split)
            else:
                final_splits.append(split)
        splits = final_splits
    
    # 빈 청크 제거 및 최소 길이 체크
    splits = [s for s in splits if s and len(s.strip()) > 50]
    
    return splits if splits else [text]

# ==================================================================
# 1-1. PDF 로딩 및 청킹
# ==================================================================
PDF_PATH = CURRENT_DIR / "ewha" / "ewha.pdf"
if not PDF_PATH.exists():
    PDF_PATH = CURRENT_DIR / "ewha.pdf"

ewha_documents = []

if PDF_PATH.exists():
    print("📄 이화여대 학칙 PDF 로딩 중...")
    loader = PyPDFLoader(str(PDF_PATH))
    docs = loader.load()
    docs = [d for d in docs if int(d.metadata.get("page", 0)) < 38] # 부칙 전까지
    
    full_text = "\n".join([d.page_content for d in docs])
    split_index = full_text.find("부칙")
    
    main_text = full_text[:split_index] if split_index != -1 else full_text
    appendix_text = full_text[split_index:] if split_index != -1 else ""
    
    print(f"📌 부칙 시작 인덱스: {split_index if split_index != -1 else '없음'}")
    print(f"📄 본문 길이: {len(main_text)}")
    print(f"📄 부칙 길이: {len(appendix_text)}")
    
    # 본문을 조항별로 분리
    print("📋 본문 조항별 분리 중...")
    main_articles = split_by_articles(main_text, is_appendix=False)
    print(f"   → {len(main_articles)}개 조항으로 분리됨")
    
    # 각 조항을 청킹 (조항이 너무 길 경우만)
    main_chunks = []
    for i, article in enumerate(main_articles):
        # 조항이 1200자 이하면 그대로 사용
        if len(article) <= 1200:
            main_chunks.append(Document(
                page_content=article,
                metadata={"source": "ewha.pdf 본문", "type": "main_text", "article_index": i}
            ))
        else:
            # 조항이 길면 세부 항목별로 분리
            text_splitter_main = RecursiveCharacterTextSplitter(
                chunk_size=1200,
                chunk_overlap=200,  # overlap 감소 (조항 경계 유지)
                separators=["\n\n①", "\n\n②", "\n\n③", "\n\n④", "\n\n⑤", "\n\n⑥", "\n\n⑦", "\n\n⑧", "\n\n⑨", "\n\n⑩",
                           "\n①", "\n②", "\n③", "\n④", "\n⑤", "\n⑥", "\n⑦", "\n⑧", "\n⑨", "\n⑩",
                           "\n\n제", "\n제", "\n\n", "\n", ". ", " "],
                length_function=len
            )
            sub_chunks = text_splitter_main.split_text(article)
            for j, chunk in enumerate(sub_chunks):
                # 각 청크의 길이 확인 (안전장치)
                if len(chunk) > 3000:  # 3000자 초과 시 경고
                    print(f"⚠️ 경고: 본문 조항 {i}의 세부 청크 {j}가 3000자를 초과합니다 ({len(chunk)}자).")
                main_chunks.append(Document(
                    page_content=chunk,
                    metadata={"source": "ewha.pdf 본문", "type": "main_text", "article_index": i, "sub_index": j}
                ))
    
    print(f"✅ 본문 청크 수: {len(main_chunks)}개")
    ewha_documents.extend(main_chunks)
    
    # 부칙을 조항별로 분리
    if appendix_text:
        print("📋 부칙 조항별 분리 중...")
        appendix_articles = split_by_articles(appendix_text, is_appendix=True)
        print(f"   → {len(appendix_articles)}개 조항으로 분리됨")
        
        appendix_chunks = []
        for i, article in enumerate(appendix_articles):
            # 부칙 조항은 더 작게 청킹 (세부 규정이 많음)
            if len(article) <= 1000:
                appendix_chunks.append(Document(
                    page_content=article,
                    metadata={"source": "ewha.pdf 부칙", "type": "appendix_text", "article_index": i}
                ))
            else:
                # 부칙 조항이 길면 세부 항목별로 분리
                text_splitter_appendix = RecursiveCharacterTextSplitter(
                    chunk_size=1000,
                    chunk_overlap=150,  # overlap 감소
                    separators=["\n\n①", "\n\n②", "\n\n③", "\n\n④", "\n\n⑤", "\n\n⑥", "\n\n⑦", "\n\n⑧", "\n\n⑨", "\n\n⑩",
                               "\n①", "\n②", "\n③", "\n④", "\n⑤", "\n⑥", "\n⑦", "\n⑧", "\n⑨", "\n⑩",
                               "별표", "\n\n제", "\n제", "\n\n", "\n", ". ", " "],
                    length_function=len
                )
                sub_chunks = text_splitter_appendix.split_text(article)
                for j, chunk in enumerate(sub_chunks):
                    # 각 청크의 길이 확인 (안전장치)
                    if len(chunk) > 3000:  # 3000자 초과 시 경고
                        print(f"⚠️ 경고: 부칙 조항 {i}의 세부 청크 {j}가 3000자를 초과합니다 ({len(chunk)}자).")
                    appendix_chunks.append(Document(
                        page_content=chunk,
                        metadata={"source": "ewha.pdf 부칙", "type": "appendix_text", "article_index": i, "sub_index": j}
                    ))
        
        print(f"✅ 부칙 청크 수: {len(appendix_chunks)}개")
        ewha_documents.extend(appendix_chunks)
    else:
        appendix_chunks = []
    
    print(f"\n📦 총 PDF 텍스트 청크: {len(main_chunks) + len(appendix_chunks) if appendix_text else len(main_chunks)}개")
    print(f"   - 본문: {len(main_chunks)}개")
    if appendix_text:
        print(f"   - 부칙: {len(appendix_chunks)}개")
    print(f"   - 최대 청크 길이: {max(len(d.page_content) for d in ewha_documents) if ewha_documents else 0}자")
else:
    print("⚠️ 경고: ewha.pdf 파일을 찾을 수 없습니다.")

# ==================================================================
# 1-2. CSV 표 데이터 처리 및 문장 생성
# ==================================================================

def coalesce(*values: object) -> str:
    """첫 번째 유효한 값을 반환"""
    for value in values:
        if value is None:
            continue
        if isinstance(value, float) and pd.isna(value):
            continue
        text = str(value).strip()
        if text:
            return text
    return ""

def build_degree_sentences_from_csv(df: pd.DataFrame, year: str = "최신 개정") -> List[str]:
    """CSV에서 학위 문장 생성"""
    sentences = []
    for idx, row in df.iterrows():
        college = coalesce(row.get("설치대학"), row.get("대학"))
        degree = coalesce(row.get("학위_종류"), row.get("학위종류"), row.get("학사"))
        major = coalesce(row.get("학과_전공"), row.get("학과 또는 전공"), row.get("전공"))
        
        if not college or not degree or not major:
            continue
            
        sentence = f"{college}의 {major} 전공은 {degree} 학위를 수여한다. ({year})"
        sentences.append(sentence)
    return sentences

def build_quota_text_v2(row: pd.Series) -> str:
    """capacity.csv -> 입학정원 문장 생성 (완전판)"""
    year = coalesce(row.get("year"))
    college = coalesce(row.get("college"))
    department = coalesce(row.get("department"))
    major = coalesce(row.get("major"))
    quota = coalesce(row.get("quota"))

    # 연도 처리
    year_str = f"{int(year)}학년도" if str(year).isdigit() else year

    # quota 정리
    if not quota or str(quota).strip().lower() == "null":
        quota = "미상"
    else:
        quota = re.sub(r'\(.*?\)|\[.*?\]', '', str(quota)).strip()

    # 케이스별 문장 생성 --------------------------------------------

    # 1) major가 있고 quota가 Null이면 → 전공별 정원 미상 케이스
    if major and quota == "미상":
        return f"{year_str} {college} {department} 소속 {major}의 입학정원은 미상입니다."

    # 2) major가 있고 quota 숫자 존재 → 전공 단위 정원
    if major and quota != "미상":
        return f"{year_str} {college} {department} 소속 {major}의 입학정원은 {quota}명입니다."

    # 3) major 없고 department 있고 quota 있음 → 학부/학과 전체 정원
    if department and quota != "미상":
        return f"{year_str} {college} {department}의 입학정원은 {quota}명입니다."

    # 4) college 전체 정원
    if "전체" in str(department):
        return f"{year_str} {college} 전체 입학정원은 {quota}명입니다."

    # 5) fallback
    return ""



def build_contract_text(row: pd.Series) -> str:
    """계약학과 텍스트 생성"""
    college = coalesce(row.get("설치대학"))
    form = coalesce(row.get("설치형태"))
    major = coalesce(row.get("학과_전공"))
    degree = coalesce(row.get("학위_종류"))
    quota = coalesce(row.get("입학정원_명"))
    period = coalesce(row.get("설치_운영기간"))
    
    text = (
        f"계약학과 설치·운영 정보입니다: 설치대학은 {college}, 설치형태는 {form}, "
        f"학과/전공은 {major}, 수여 학위는 {degree}이며, "
        f"입학 정원은 {quota}명입니다. 설치·운영 기간은 {period}입니다."
    )
    return text

def build_grade_text(row: pd.Series) -> str:
    """성적점 텍스트 생성"""
    year = coalesce(row.get("year"))
    grade = coalesce(row.get("grade"))
    gpa = coalesce(row.get("gpa"))
    
    if not year or not grade or pd.isna(gpa):
        return ""
    
    # year가 "from 1980"이면 "1980학년도 이후", "before 1980"이면 "1980학년도 이전"
    if "from 1980" in str(year).lower() or "1980 이후" in str(year):
        year_desc = "1980학년도 이후 입학생"
    elif "before 1980" in str(year).lower() or "1980 이전" in str(year):
        year_desc = "1980학년도 이전 입학생"
    else:
        year_desc = str(year)
    
    return f"{year_desc}에 적용하는 등급 {grade}의 성적점은 {gpa}점입니다."

# CSV 파일 처리
csv_documents = []

# CSV 파일 경로 설정 (ewha csvs 폴더 우선)
csvs_dir = CURRENT_DIR / "ewha" / "ewha csvs"
if not csvs_dir.exists():
    csvs_dir = CURRENT_DIR / "ewha csvs"  # 하위 호환성: 프로젝트 루트에 있을 경우
if not csvs_dir.exists():
    csvs_dir = CURRENT_DIR  # 폴더가 없으면 현재 디렉토리 사용

# 1) degrees 관련 파일 처리
degrees_dir = csvs_dir / "degrees" if (csvs_dir / "degrees").exists() else CURRENT_DIR / "degrees"
degree_sentences_file = csvs_dir / "degree_sentences_all.csv의 사본" if (csvs_dir / "degree_sentences_all.csv의 사본").exists() else degrees_dir / "degree_sentences_all.csv의 사본"
degree_latest_file = csvs_dir / "degree_latest.csv의 사본" if (csvs_dir / "degree_latest.csv의 사본").exists() else degrees_dir / "degree_latest.csv의 사본"

# 우선순위 1: 이미 문장으로 변환된 파일 사용
if degree_sentences_file.exists():
    df_sentences = pd.read_csv(degree_sentences_file, encoding='utf-8-sig')
    for _, row in df_sentences.iterrows():
        sentence = str(row.get("sentence", "")).strip()
        if sentence:
            csv_documents.append(Document(
                page_content=sentence,
                metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
            ))
    print(f"✅ degree_sentences_all.csv 처리 완료: {len(df_sentences)}개 문장")
# 우선순위 2: degree_latest.csv를 사용하여 문장 생성
elif degree_latest_file.exists():
    df_degrees = pd.read_csv(degree_latest_file, encoding='utf-8-sig')
    df_degrees = df_degrees.dropna(subset=['대학', '학사', '전공'], how='all')
    degree_sentences = build_degree_sentences_from_csv(df_degrees, "최신 개정")
    for sentence in degree_sentences:
        csv_documents.append(Document(
            page_content=sentence,
            metadata={"source": "[별표 2] 학사학위의 종류", "page": 51, "type": "degree"}
        ))
    print(f"✅ degree_latest.csv 처리 완료: {len(degree_sentences)}개 문장")
else:
    print("⚠️ degrees 관련 파일을 찾을 수 없습니다.")

# 2) capacity.csv 처리 (입학정원 데이터)
capacity_path = csvs_dir / "capacity.csv"
if not capacity_path.exists():
    capacity_path = CURRENT_DIR / "capacity.csv"

if capacity_path.exists():
    df_capacity = pd.read_csv(capacity_path, encoding='utf-8-sig')

    for _, row in df_capacity.iterrows():
        text = build_quota_text_v2(row)
        if text:
            csv_documents.append(Document(
                page_content=text,
                metadata={
                    "source": f"[입학정원] {row.get('year')}",
                    "type": "quota",
                    "year": row.get("year"),
                    "college": row.get("college"),
                    "major": row.get("major")
                }
            ))
    print("capacity 처리 완료:", len([d for d in csv_documents if d.metadata.get('type')=='quota']))
else:
    print("⚠️ capacity.csv 파일이 없습니다.")

# 3) contract_dept.csv 처리
contract_path = csvs_dir / "contract_dept.csv"
if not contract_path.exists():
    contract_path = CURRENT_DIR / "contract_dept.csv"

if contract_path.exists():
    df_contract = pd.read_csv(contract_path, encoding='utf-8-sig')
    for _, row in df_contract.iterrows():
        text = build_contract_text(row)
        if text and text.strip():
            csv_documents.append(Document(
                page_content=text,
                metadata={"source": "[별표 3] 계약학과 설치·운영", "page": 53, "type": "contract"}
            ))
    print(f"✅ contract_dept.csv 처리 완료: {len([d for d in csv_documents if d.metadata.get('type') == 'contract'])}개 문서")
else:
    print("⚠️ contract_dept.csv 파일이 없습니다.")

# 4) grade.csv 처리 (1980년 기준 성적점 데이터)
grade_path = csvs_dir / "grade.csv"
if not grade_path.exists():
    grade_path = CURRENT_DIR / "grade.csv"

if grade_path.exists():
    df_grade = pd.read_csv(grade_path, encoding='utf-8-sig')
    for _, row in df_grade.iterrows():
        text = build_grade_text(row)
        if text and text.strip():
            csv_documents.append(Document(
                page_content=text,
                metadata={"source": "[별표] 성적점 등급표", "page": 0, "type": "grade"}
            ))
    print(f"✅ grade.csv 처리 완료: {len([d for d in csv_documents if d.metadata.get('type') == 'grade'])}개 문서")
else:
    print("⚠️ grade.csv 파일이 없습니다.")


print(f"\n📦 총 CSV 문서: {len(csv_documents)}개")

# 최종적으로 이화여대 문서 합치기
ewha_full_docs = ewha_documents + csv_documents
print(f"\n📚 이화여대 로컬 DB 준비 완료: 총 {len(ewha_full_docs)}개 문서")
print(f"   - PDF 텍스트: {len(ewha_documents)}개")
print(f"   - CSV 표 데이터: {len(csv_documents)}개")

In [ ]:
# ==================================================================
# 2. 이화여대 전용 Vector Store 생성 (Local Knowledge Base)
# ==================================================================

embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

# Windows 한글 경로 문제 해결: 임시 디렉토리 사용 (RAG_통합.ipynb 방식)
# FAISS의 C++ 백엔드가 한글 경로를 처리하지 못하는 문제 해결
import platform
import tempfile

if platform.system() == 'Windows':
    # Windows: 환경 변수 TEMP 사용 (보통 C:\Users\USERNAME\AppData\Local\Temp)
    temp_dir = Path(tempfile.gettempdir())
    # 프로젝트별 고유한 디렉토리명 생성
    vector_db_base = temp_dir / "rag_ewha_vectorstore"
    vector_db_base.mkdir(parents=True, exist_ok=True)
    EWHA_VECTOR_DB_PATH_STR = str(vector_db_base.resolve())
    EWHA_VECTOR_DB_PATH = vector_db_base
    print(f"📂 Windows 한글 경로 문제 해결: 임시 디렉토리 사용")
    print(f"📂 벡터 DB 경로: {EWHA_VECTOR_DB_PATH_STR}")
else:
    # Windows가 아닌 경우 상대 경로 사용
    EWHA_VECTOR_DB_PATH = CURRENT_DIR / "vectorstore_ewha"
    EWHA_VECTOR_DB_PATH.mkdir(parents=True, exist_ok=True)
    EWHA_VECTOR_DB_PATH_STR = str(EWHA_VECTOR_DB_PATH.resolve())

ewha_vector_store = None

vector_db_dir = Path(EWHA_VECTOR_DB_PATH_STR)

if vector_db_dir.exists() and (vector_db_dir / "index.faiss").exists():
    try:
        ewha_vector_store = FAISS.load_local(
            folder_path=EWHA_VECTOR_DB_PATH_STR,
            embeddings=embeddings,
            allow_dangerous_deserialization=True
        )
        print("📂 기존 이화여대 벡터 스토어 로드 완료")
    except Exception as e:
        print(f"⚠️ 로드 실패, 새로 생성합니다. (에러: {e})")
        ewha_vector_store = None

if ewha_vector_store is None and ewha_full_docs:
    # ⚠️ 안전장치: 각 문서의 길이 확인 (4000 토큰 = 약 3000자 제한)
    MAX_CHUNK_LENGTH = 3000  # 안전하게 3000자로 설정
    filtered_docs = []
    
    for doc in ewha_full_docs:
        if len(doc.page_content) > MAX_CHUNK_LENGTH:
            # 문서가 너무 길면 추가로 분할
            print(f"⚠️ 문서가 {len(doc.page_content)}자로 너무 깁니다. 추가 분할 중...")
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=MAX_CHUNK_LENGTH - 200,  # overlap을 위해 200자 여유
                chunk_overlap=100,
                separators=["\n\n", "\n", ". ", " "],
                length_function=len
            )
            sub_chunks = text_splitter.split_text(doc.page_content)
            for i, chunk in enumerate(sub_chunks):
                if len(chunk) <= MAX_CHUNK_LENGTH:
                    new_doc = Document(
                        page_content=chunk,
                        metadata={**doc.metadata, "sub_chunk_index": i}
                    )
                    filtered_docs.append(new_doc)
                else:
                    print(f"  ⚠️ 추가 분할 후에도 청크가 {len(chunk)}자입니다. 더 작게 분할합니다.")
                    # 재귀적으로 더 작게 분할
                    text_splitter_small = RecursiveCharacterTextSplitter(
                        chunk_size=1000,
                        chunk_overlap=100,
                        separators=["\n\n", "\n", ". ", " "],
                        length_function=len
                    )
                    tiny_chunks = text_splitter_small.split_text(chunk)
                    for j, tiny_chunk in enumerate(tiny_chunks):
                        new_doc = Document(
                            page_content=tiny_chunk,
                            metadata={**doc.metadata, "sub_chunk_index": i, "tiny_chunk_index": j}
                        )
                        filtered_docs.append(new_doc)
        else:
            filtered_docs.append(doc)
    
    print(f"🔄 이화여대 벡터 스토어 생성 중... (총 {len(filtered_docs)}개 문서)")
    print(f"   - 최대 문서 길이: {max(len(d.page_content) for d in filtered_docs) if filtered_docs else 0}자")
    
    # 배치 처리로 임베딩 (메모리 절약)
    try:
        ewha_vector_store = FAISS.from_documents(filtered_docs, embeddings)
        # RAG_통합.ipynb 방식: 문자열 경로 사용 (임시 디렉토리는 영문 경로)
        ewha_vector_store.save_local(EWHA_VECTOR_DB_PATH_STR)
        print("✅ 생성 완료")
    except Exception as e:
        print(f"❌ 벡터 스토어 생성 실패: {e}")
        print("   긴 문서를 찾아서 더 작게 분할하는 중...")
        # 실패 시 모든 문서를 1000자로 강제 분할
        final_docs = []
        for doc in filtered_docs:
            if len(doc.page_content) > 1000:
                text_splitter_final = RecursiveCharacterTextSplitter(
                    chunk_size=1000,
                    chunk_overlap=100,
                    separators=["\n\n", "\n", ". ", " "],
                    length_function=len
                )
                chunks = text_splitter_final.split_text(doc.page_content)
                for i, chunk in enumerate(chunks):
                    final_docs.append(Document(
                        page_content=chunk,
                        metadata={**doc.metadata, "final_chunk_index": i}
                    ))
            else:
                final_docs.append(doc)
        
        print(f"   최종 문서 수: {len(final_docs)}개")
        ewha_vector_store = FAISS.from_documents(final_docs, embeddings)
        # RAG_통합.ipynb 방식: 문자열 경로 사용 (임시 디렉토리는 영문 경로)
        ewha_vector_store.save_local(EWHA_VECTOR_DB_PATH_STR)
        print("✅ 생성 완료 (강제 분할 적용)")

# 이화여대 전용 Retriever
ewha_retriever = ewha_vector_store.as_retriever(search_kwargs={"k": 10})

In [4]:
# ==================================================================
# 3. 사전 벡터화 Wikipedia 문서 추가 (NLTK 기반 키워드 추출)
# ==================================================================

import nltk
import time 

# NLTK 리소스 (한 번만 다운로드, 이후엔 캐시 사용)
try:
    nltk.download('punkt', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
except:
    pass

# --------------------------------------------------
# 3-0. 질문에서 Wikipedia 검색용 키워드 추출 (NLTK 기반)
# --------------------------------------------------

def extract_wiki_terms(question: str, max_terms: int = 3) -> List[str]:
    """
    질문에서 Wikipedia 검색용 후보 키워드를 추출한다.
    - NLTK 기반 고유명사(NNP), 길이가 긴 명사 우선
    - 도메인별 정교화된 키워드 매핑 적용
    - 불필요한 단어 필터링 (QUESTION, QUESTION26 등)
    """
    q = question.lower()

    # 불필요한 단어 필터링 리스트
    stop_words = {'question', 'questions', 'question26', 'question27', 'question28', 'question29', 
                  'question30', 'following', 'information', 'refers', 'refers to', 'this', 'that',
                  'which', 'what', 'who', 'when', 'where', 'why', 'how', 'the', 'a', 'an',
                  'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
                  'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might',
                  'can', 'cannot', 'must', 'shall', 'all', 'some', 'any', 'none', 'both',
                  'each', 'every', 'other', 'another', 'such', 'same', 'different', 'various'}

    # 1️⃣ 도메인 기반 우선 처리 맵 (정밀 매핑)
    topic_map = [
        # 철학
        (["kohlberg"], ["Kohlberg's stages of moral development"]),
        (["moral development"], ["Kohlberg's stages of moral development"]),
        (["kant", "imperative"], ["Categorical imperative", "Immanuel Kant"]),
        (["utilitarian", "pleasure", "pain"], ["Utilitarianism", "Jeremy Bentham"]),
        (["positivism"], ["Legal positivism"]),

        # 형법 / 범죄
        (["robbery"], ["Robbery"]),
        (["larceny"], ["Larceny"]),
        (["burglary"], ["Burglary"]),
        (["stolen", "receiving"], ["Receiving stolen property"]),

        # Singer 영역
        (["singer", "affluence", "famine"], ["Famine, Affluence and Morality", "Peter Singer"]),
    ]

    # 1️⃣ 도메인 기반 맵핑 그대로
    for triggers, wiki_titles in topic_map:
        if any(t in q for t in triggers):
            return wiki_titles[:max_terms]

    # 2️⃣ NLTK noun 기반 fallback
    try:
        tokens = nltk.word_tokenize(question)
        tagged = nltk.pos_tag(tokens)
    except Exception:
        tokens = re.findall(r"[A-Za-z][A-Za-z\-]{2,}", question)
        filtered_tokens = [t for t in tokens if t.lower() not in stop_words]
        return filtered_tokens[:max_terms]

    candidates = [
        word for word, pos in tagged
        if pos.startswith("NNP") or (pos.startswith("NN") and len(word) >= 6)
    ]

    cleaned = []
    for w in candidates:
        w = re.sub(r"[^A-Za-z\-]", "", w)
        if len(w) >= 3 and w.lower() not in stop_words:
            cleaned.append(w)

    unique = []
    seen = set()
    for w in cleaned:
        wl = w.lower()
        if wl not in seen:
            seen.add(wl)
            unique.append(w)

    return unique[:max_terms]




# --------------------------------------------------
# 3-1. Wikipedia 문서 가져오기 함수
# --------------------------------------------------

# 실패한 페이지에 대한 대체 제목 매핑
TITLE_ALTERNATIVES = {
    "Emergency ethics": ["Medical ethics", "Emergency medical ethics", "Bioethics", "Clinical ethics"],
    "Arctic archaeology": ["Archaeology of the Arctic", "Arctic Archaeology", "Prehistory of Alaska", "Alaska archaeology"],
    "Denali complex": ["Denali National Park", "Denali", "Mount Denali", "Alaska Range"],
    "Nenana complex": ["Nenana Valley", "Nenana River", "Alaska archaeology", "Prehistory of Alaska"]
}

def generate_title_variations(title: str) -> List[str]:
    """제목의 다양한 변형 생성"""
    variations = [title]  # 원본 제목
    
    # 대체 제목 매핑 확인
    if title in TITLE_ALTERNATIVES:
        variations.extend(TITLE_ALTERNATIVES[title])
    
    # 제목 변형 생성
    # 1. 첫 글자 대문자 변형
    if title != title.capitalize():
        variations.append(title.capitalize())
    
    # 2. 단어별 첫 글자 대문자
    title_title = title.title()
    if title_title != title:
        variations.append(title_title)
    
    # 3. 단어 순서 변경 (2단어인 경우)
    words = title.split()
    if len(words) == 2:
        variations.append(f"{words[1]} {words[0]}")
    
    # 중복 제거 및 순서 유지
    seen = set()
    unique_variations = []
    for var in variations:
        if var not in seen:
            seen.add(var)
            unique_variations.append(var)
    
    return unique_variations

def fetch_wikipedia_article(title: str, language: str = "en") -> Optional[str]:
    
    
    max_retries=3
    retry_delay=2
    for attempt in range(max_retries):
        try:
            # wikipediaapi 사용
            wiki_wiki = wikipediaapi.Wikipedia(
                user_agent='MyNLPProject/1.0',
                language=language,
                extract_format=wikipediaapi.ExtractFormat.WIKI
            )
                    
            # 제목 변형 목록 생성
            title_variations = generate_title_variations(title)
            
            # 각 제목 변형 시도
            for attempt_title in title_variations:
                page = wiki_wiki.page(attempt_title)
                
                if page.exists():
                    # 동음이의어 페이지 확인
                    text_lower = page.text.lower()
                    if "may refer to" in text_lower or "disambiguation" in text_lower:
                        # 동음이의어 페이지인 경우, 첫 번째 링크 시도
                        if page.links:
                            first_link_title = list(page.links.keys())[0]
                            if attempt_title != title:
                                print(f"   🔄 대체 제목 성공: '{title}' → '{attempt_title}' (동음이의어 → '{first_link_title}')")
                            else:
                                print(f"   🔄 동음이의어 페이지 감지: '{title}' → '{first_link_title}' 시도")
                            disambiguation_page = wiki_wiki.page(first_link_title)
                            if disambiguation_page.exists():
                                return disambiguation_page.text
                        # 링크가 없으면 원본 페이지 텍스트 반환
                        if attempt_title != title:
                            print(f"   🔄 대체 제목 성공: '{title}' → '{attempt_title}'")
                        return page.text
                    else:
                        # 일반 페이지
                        if attempt_title != title:
                            print(f"   🔄 대체 제목 성공: '{title}' → '{attempt_title}'")
                        return page.text
            
                    # 모든 변형 실패
            print(f"⚠️ Wikipedia 문서 '{title}' 가져오기 실패 (모든 변형 시도 실패)")
            return None
                
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"   ⚠️ 시도 {attempt + 1}/{max_retries} 실패: {type(e).__name__}, {retry_delay}초 후 재시도...")
                time.sleep(retry_delay)
            else:
                print(f"   ❌ 최종 실패: {type(e).__name__}: {str(e)[:100]}")
                return None
def create_wikipedia_document(title: str, content: str, priority: str = "high") -> List[Document]:
    """Wikipedia 문서를 청크로 분할"""
    if not content:
        return []

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=120,
        separators=["\n\n", "\n", ". ", " "]
    )

    chunks = text_splitter.split_text(content)
    documents = []
    for i, chunk in enumerate(chunks):
        documents.append(Document(
            page_content=chunk,
            metadata={
                "source": f"Wikipedia: {title}",
                "type": "wikipedia",
                "priority": priority,
                "chunk_index": i
            }
        ))

    return documents


# --------------------------------------------------
# 3-2. 사전 정의된 Wikipedia 문서 목록
# --------------------------------------------------

# 최우선 Wikipedia 문서 목록 (Phase 1)
priority_1_wiki_titles = {
    "Kohlberg's stages of moral development": "high",
    "Robbery": "high",
    "Larceny": "high",
    "Burglary": "high",
    "Homo erectus": "high",
    "Neoteny": "high",
    # 오답 분석 기반 추가 문서
    "Alan Dershowitz": "high",
    "Emergency ethics": "high",
    "Thomas Aquinas": "high",
    "Natural law": "high",
    "Unnatural acts": "high",
    "Negligence": "high",
    "Vicarious liability": "high",
    "Tort law": "high",
    "Assault (tort)": "high",
    "Battery (tort)": "high",
    "Paternity test": "high",
    "Evidence (law)": "high",
    "Strict liability": "high",
    "Product liability": "high",
}

# 차순위 Wikipedia 문서 목록 (Phase 2)
priority_2_wiki_titles = {
    "Tang dynasty": "medium",
    "Deutschland über alles": "medium",
    "Bipolar disorder": "medium",
    "Ethics in psychology": "medium",
    "Utilitarianism": "medium",
    "Categorical imperative": "medium",
    "Jurisprudence": "medium",
    "Famine, Affluence and Morality": "medium",
}

# 보조 Wikipedia 문서 목록 (Phase 3)
priority_3_wiki_titles = {
    "Human evolution": "low",
    "Homo habilis": "low",
    "Australopithecus afarensis": "low",
    "Aristotelianism": "low",
    "Peter Singer": "low",
    "Green marketing": "low",
    "Theory of multiple intelligences": "low",
    "Approach-avoidance conflict": "low",
    "Receiving stolen property": "low",
    "Legal positivism": "low",
    "John Frere": "low",
    "Du Fu": "low",
    "Enforcement of foreign judgments": "low",
    # 오답 분석 기반 추가 문서
    "Lithic technology": "low",
    "Microblade technology": "low",
    "Arctic archaeology": "low",
    "Denali complex": "low",
    "Nenana complex": "low",
    "Divorce": "low",
    "Child custody": "low",
    "Family therapy": "low",
}

# --------------------------------------------------
# 3-3. Wikipedia 문서 벡터화 및 통합
# --------------------------------------------------

# 벡터화할 문서 선택 (기본값: 전체 적용)
WIKI_PHASE = 3  # 1: 최우선, 2: 최우선+차순위, 3: 전체

wiki_documents = []

if WIKI_PHASE >= 1:
    print("\n📖 Phase 1: 최우선 Wikipedia 문서 벡터화 중...")
    for title, priority in priority_1_wiki_titles.items():
        print(f"   - {title} 다운로드 중...")
        content = fetch_wikipedia_article(title)
        if content:
            docs = create_wikipedia_document(title, content, priority)
            wiki_documents.extend(docs)
            print(f"     ✅ {len(docs)}개 청크 생성")
        else:
            print(f"     ❌ 다운로드 실패")

if WIKI_PHASE >= 2:
    print("\n📖 Phase 2: 차순위 Wikipedia 문서 벡터화 중...")
    for title, priority in priority_2_wiki_titles.items():
        print(f"   - {title} 다운로드 중...")
        content = fetch_wikipedia_article(title)
        if content:
            docs = create_wikipedia_document(title, content, priority)
            wiki_documents.extend(docs)
            print(f"     ✅ {len(docs)}개 청크 생성")
        else:
            print(f"     ❌ 다운로드 실패")

if WIKI_PHASE >= 3:
    print("\n📖 Phase 3: 보조 Wikipedia 문서 벡터화 중...")
    for title, priority in priority_3_wiki_titles.items():
        print(f"   - {title} 다운로드 중...")
        content = fetch_wikipedia_article(title)
        if content:
            docs = create_wikipedia_document(title, content, priority)
            wiki_documents.extend(docs)
            print(f"     ✅ {len(docs)}개 청크 생성")
        else:
            print(f"     ❌ 다운로드 실패")


# --------------------------------------------------
# 3-4. 통합 벡터 스토어 생성 (이화여대 + Wikipedia)
# --------------------------------------------------

# 모든 문서 통합
all_documents_for_vectorstore = []

# 이화여대 문서 추가
if ewha_full_docs:
    all_documents_for_vectorstore.extend(ewha_full_docs)
    print(f"\n📚 이화여대 문서: {len(ewha_full_docs)}개 청크")

# Wikipedia 문서 추가
if wiki_documents:
    all_documents_for_vectorstore.extend(wiki_documents)
    print(f"📚 Wikipedia 문서: {len(wiki_documents)}개 청크 추가")

print(f"\n📚 총 통합 문서: {len(all_documents_for_vectorstore)}개")

# --------------------------------------------------
# 3-6. 통합 벡터 스토어 생성 또는 로드
# --------------------------------------------------

# Windows 한글 경로 문제 해결
import platform
import tempfile

if platform.system() == 'Windows':
    temp_dir = Path(tempfile.gettempdir())
    vector_db_base = temp_dir / "rag_ewha_vectorstore_with_wiki"
    vector_db_base.mkdir(parents=True, exist_ok=True)
    INTEGRATED_VECTOR_DB_PATH_STR = str(vector_db_base.resolve())
    INTEGRATED_VECTOR_DB_PATH = vector_db_base
    print(f"📂 통합 벡터 DB 경로: {INTEGRATED_VECTOR_DB_PATH_STR}")
else:
    INTEGRATED_VECTOR_DB_PATH = CURRENT_DIR / "vectorstore_integrated"
    INTEGRATED_VECTOR_DB_PATH.mkdir(parents=True, exist_ok=True)
    INTEGRATED_VECTOR_DB_PATH_STR = str(INTEGRATED_VECTOR_DB_PATH.resolve())

integrated_vector_store = None
vector_db_dir = Path(INTEGRATED_VECTOR_DB_PATH_STR)

# 통합 벡터 스토어 로드 시도
FORCE_REBUILD = True  # True로 설정하면 기존 벡터 스토어 무시하고 재생성 (새로 추가한 Wikipedia 문서 반영)

if not FORCE_REBUILD and vector_db_dir.exists() and (vector_db_dir / "index.faiss").exists():
    try:
        integrated_vector_store = FAISS.load_local(
            folder_path=INTEGRATED_VECTOR_DB_PATH_STR,
            embeddings=embeddings,
            allow_dangerous_deserialization=True
        )
        print("📂 기존 통합 벡터 스토어 로드 완료 (Wikipedia 포함)")
    except Exception as e:
        print(f"⚠️ 로드 실패, 새로 생성합니다. (에러: {e})")
        integrated_vector_store = None

# 벡터 스토어가 없거나 재생성해야 하는 경우
if integrated_vector_store is None or FORCE_REBUILD:
    if all_documents_for_vectorstore:
        print(f"\n⏳ 통합 벡터 스토어 생성 중... (총 {len(all_documents_for_vectorstore)}개 문서)")
        
        # 문서 길이 체크 및 필터링
        MAX_CHUNK_LENGTH = 3000
        filtered_docs = []
        
        for doc in all_documents_for_vectorstore:
            if len(doc.page_content) > MAX_CHUNK_LENGTH:
                print(f"⚠️ 문서가 {len(doc.page_content)}자로 너무 깁니다. 추가 분할 중...")
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=MAX_CHUNK_LENGTH - 200,
                    chunk_overlap=100,
                    separators=["\n\n", "\n", ". ", " "],
                    length_function=len
                )
                sub_chunks = text_splitter.split_text(doc.page_content)
                for i, chunk in enumerate(sub_chunks):
                    if len(chunk) <= MAX_CHUNK_LENGTH:
                        new_doc = Document(
                            page_content=chunk,
                            metadata={**doc.metadata, "sub_chunk_index": i}
                        )
                        filtered_docs.append(new_doc)
                    else:
                        # 재귀적으로 더 작게 분할
                        text_splitter_small = RecursiveCharacterTextSplitter(
                            chunk_size=1000,
                            chunk_overlap=100,
                            separators=["\n\n", "\n", ". ", " "],
                            length_function=len
                        )
                        tiny_chunks = text_splitter_small.split_text(chunk)
                        for j, tiny_chunk in enumerate(tiny_chunks):
                            new_doc = Document(
                                page_content=tiny_chunk,
                                metadata={**doc.metadata, "sub_chunk_index": i, "tiny_chunk_index": j}
                            )
                            filtered_docs.append(new_doc)
            else:
                filtered_docs.append(doc)
        
        print(f"   최종 문서 수: {len(filtered_docs)}개")
        
        try:
            integrated_vector_store = FAISS.from_documents(filtered_docs, embeddings)
            integrated_vector_store.save_local(INTEGRATED_VECTOR_DB_PATH_STR)
            print(f"✅ 통합 벡터 스토어 생성 완료: {INTEGRATED_VECTOR_DB_PATH_STR}")
        except Exception as e:
            print(f"❌ 벡터 스토어 생성 실패: {e}")
            print("   더 작은 청크로 재시도 중...")
            # 실패 시 강제 분할
            final_docs = []
            for doc in filtered_docs:
                if len(doc.page_content) > 1000:
                    text_splitter_final = RecursiveCharacterTextSplitter(
                        chunk_size=1000,
                        chunk_overlap=100,
                        separators=["\n\n", "\n", ". ", " "],
                        length_function=len
                    )
                    chunks = text_splitter_final.split_text(doc.page_content)
                    for i, chunk in enumerate(chunks):
                        final_docs.append(Document(
                            page_content=chunk,
                            metadata={**doc.metadata, "final_chunk_index": i}
                        ))
                else:
                    final_docs.append(doc)
            
            print(f"   최종 문서 수: {len(final_docs)}개")
            integrated_vector_store = FAISS.from_documents(final_docs, embeddings)
            integrated_vector_store.save_local(INTEGRATED_VECTOR_DB_PATH_STR)
            print("✅ 생성 완료 (강제 분할 적용)")
    else:
        print("⚠️ 통합할 문서가 없습니다.")

# 통합 벡터 스토어 (MMLU 문제용 - Wikipedia 포함)
# EWHA 문제는 기존 ewha_vector_store만 사용 (이화여대 문서만)
# MMLU 문제는 integrated_vector_store 사용 (Wikipedia 문서 포함)

if integrated_vector_store is not None:
    print("\n✅ 통합 벡터 스토어 준비 완료 (Wikipedia 포함)")
    print("   - EWHA 문제: 이화여대 벡터 스토어만 사용")
    print("   - MMLU 문제: 통합 벡터 스토어 사용 (이화여대 + Wikipedia)")
    # MMLU용 retriever 생성 (통합 벡터 스토어 사용)
    mmlu_retriever = integrated_vector_store.as_retriever(search_kwargs={"k": 4})
else:
    print("\n⚠️ 통합 벡터 스토어가 없습니다.")
    mmlu_retriever = None

# ewha_retriever는 기존 이화여대 벡터 스토어만 사용 (변경 없음)
print("✅ NLTK 기반 Wikipedia 검색 준비 완료")


✅ 통합 벡터 스토어 생성 완료: C:\Users\user\AppData\Local\Temp\rag_ewha_vectorstore_with_wiki

✅ 통합 벡터 스토어 준비 완료 (Wikipedia 포함)
   - EWHA 문제: 이화여대 벡터 스토어만 사용
   - MMLU 문제: 통합 벡터 스토어 사용 (이화여대 + Wikipedia)
✅ NLTK 기반 Wikipedia 검색 준비 완료
